In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [ ]:
train_df = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/train.csv')
submission_df = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
submission_df.head()

In [ ]:
print(f"Images in train index file: {train_df.image.nunique()}")
print(f"Species in train index file: {train_df.species.nunique()}")
print(f"Individual IDs in train index file: {train_df.individual_id.nunique()}")

print(f"Images in train images folder: {len(os.listdir('/kaggle/input/happy-whale-and-dolphin/train_images'))}")
print(f"Images in test images folder: {len(os.listdir('/kaggle/input/happy-whale-and-dolphin/test_images'))}")

In [ ]:
print("Top 10 individual_id")
train_df.individual_id.value_counts().head(10)

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.kdeplot(np.log(train_df.individual_id.value_counts()))
plt.title("Logaritmic distribution of individual_id frequency in images")
plt.show()

In [ ]:
temp = train_df["species"].value_counts()
df = pd.DataFrame({'Species': temp.index,
                   'Images': temp.values
                  })
plt.figure(figsize = (12,6))
plt.title('Species distribution - images per each species - train dataset')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Species', y="Images", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
# ここで、各種ごとにいくつの個別IDがあるかを確認

temp = train_df.groupby(["species"])["individual_id"].nunique()
df = pd.DataFrame({'Species': temp.index,
                   'Unique ID Count': temp.values
                  })
df = df.sort_values(['Unique ID Count'], ascending=False)
plt.figure(figsize = (12,6))
plt.title('Species distribution - Individual IDs per each species - train dataset')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Species', y="Unique ID Count", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:

train_df_list = list(train_df.image.unique())
train_images_list = list(os.listdir('/kaggle/input/happy-whale-and-dolphin/train_images'))
delta = set(train_df_list) & set(train_images_list)
minus = set(train_df_list) - set(train_images_list)
print(f"Images in train dataset: {len(train_df_list)}\nImages in train folder: {len(train_images_list)}\nIntersection: {len(delta)}\nDifference: {len(minus)}")

In [ ]:
def read_image_sizes(file_name):
    image = cv2.imread('/kaggle/input/happy-whale-and-dolphin/train_images/' + file_name)
    return list(image.shape)

In [ ]:
m = np.stack(train_df['image'].sample(1000).apply(read_image_sizes))
df = pd.DataFrame(m,columns=['w','h','c'])

In [ ]:
def plot_image_samples(species):
    root_path = "/kaggle/input/happy-whale-and-dolphin/"
    fig.subplots_adjust(hspace = .1, wspace=.1)
    df = train_df[train_df['species']==species].copy()
    df.index = range(len(df.index))
    
    f, ax = plt.subplots(4, 4, figsize=(16,16))

    for i in range(16):
        file = df.loc[i, 'image']
        species = df.loc[i, 'species']
        identifier = df.loc[i, 'individual_id']
        img = cv2.imread(root_path+'train_images/'+file)
        ax[i//4, i%4].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax[i//4, i%4].set_title(identifier+" ("+species+")")
        ax[i//4, i%4].axis('off')

In [ ]:
plot_image_samples("bottlenose_dolphin")

In [ ]:
def rotate_values(x):
    xcopy = x.split()
    temp = xcopy[4]
    xcopy[4] = xcopy[0]
    xcopy[0] = temp
    xcopy = " ".join(xcopy)
    return xcopy

In [ ]:
def new_indivisual_out(x):
    xcopy = x.split()
    return xcopy[4]

In [ ]:
submission_df["predictions"] = submission_df["predictions"].apply(lambda x: new_indivisual_out(x))

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)